In [17]:
import pickle
import random
import eng_to_ipa as ipa

In [18]:
dataset_file = open('dataset.txt')

dataset = dataset_file.readlines()
dataset = [word.strip('\n') for word in dataset]
random.shuffle(dataset)

In [19]:
training_data_ipa = open('training_data_ipa.txt', 'w+', encoding = 'utf-8')
metadata_ipa_file = open('metadata_ipa.txt', 'w+')

ipa_max_encoder_len = 0
ipa_max_decoder_len = 0

data_counter = 0

existing_syl = []

ipa_encoder_vocab = []
ipa_decoder_vocab = []

def ipa_splitter(line):
    global training_data_ipa
    global ipa_max_encoder_len
    global ipa_max_decoder_len
    global ipa_encoder_vocab
    global ipa_decoder_vocab
    global existing_syl
    global data_counter
    syllables = line.split(';')
    for syl in syllables:
        if syl not in existing_syl:
            existing_syl += [syl]
            translation = ipa.convert(syl)
            if not translation.endswith('*'):
                data_counter += 1
                translation = '<' + translation + '>'
                ipa_max_encoder_len = max(len(syl), ipa_max_encoder_len)
                ipa_max_decoder_len = max(len(translation), ipa_max_decoder_len)
                training_data_ipa.write(syl + '\t' + translation + '\n')
                for c in syl:
                    if c not in ipa_encoder_vocab:
                        ipa_encoder_vocab += [c]
                for i in translation:
                    if i not in ipa_decoder_vocab:
                        ipa_decoder_vocab += [i]

In [20]:
# creating syllable to ipa data
for line in dataset:
    ipa_splitter(line)

ipa_encoder_vocab = sorted(ipa_encoder_vocab)
ipa_decoder_vocab = sorted(ipa_decoder_vocab)

ipa_encoder_to_int = dict((a,i) for i,a in enumerate(ipa_encoder_vocab, 1))
ipa_int_to_encoder = dict((i,a) for i,a in enumerate(ipa_encoder_vocab, 1))

ipa_decoder_to_int = dict((a,i) for i,a in enumerate(ipa_decoder_vocab, 1))
ipa_int_to_decoder = dict((i,a) for i,a in enumerate(ipa_decoder_vocab, 1))

ipa_num_encoder_vocab = len(ipa_encoder_vocab) + 1
ipa_num_decoder_vocab = len(ipa_decoder_vocab) + 1    

training_data_ipa.close()
metadata_ipa_file.write("ipa_max_encoder_len: " + '\t' + str(ipa_max_encoder_len) + '\n')
metadata_ipa_file.write("ipa_max_decoder_len: " + '\t' + str(ipa_max_decoder_len) + '\n')
metadata_ipa_file.write("ipa_num_encoder_vocab: " + '\t' + str(ipa_num_encoder_vocab) + '\n')
metadata_ipa_file.write("ipa_num_decoder_vocab: " + '\t' + str(ipa_num_decoder_vocab) + '\n')
metadata_ipa_file.close()

print(data_counter)

4893


In [21]:
e2i_file = open('ipa_e2i.pkl' , 'wb')
i2e_file = open('ipa_i2e.pkl' , 'wb')
d2i_file = open('ipa_d2i.pkl' , 'wb')
i2d_file = open('ipa_i2d.pkl' , 'wb')

pickle.dump(ipa_encoder_to_int, e2i_file)
pickle.dump(ipa_int_to_encoder, i2e_file)
pickle.dump(ipa_decoder_to_int, d2i_file)
pickle.dump(ipa_int_to_decoder, i2d_file)

e2i_file.close()
i2e_file.close()
d2i_file.close()
i2d_file.close()